<img align="left" src="https://lever-client-logos.s3.amazonaws.com/864372b1-534c-480e-acd5-9711f850815c-1524247202159.png" width=200>
<br></br>
<br></br>

# Neural Networks

## *Data Science Unit 4 Sprint 2 Assignment 1*

## Define the Following:
You can add image, diagrams, whatever you need to ensure that you understand the concepts below.

### Input Layer: provide infomration to the network. There isnt any computation performed in the input nodes. These simply pass information to the next layer. 
### Hidden Layer: This layer performs computations on the information that is passed to them from the input layer. Once the computations have been done on the input information that information is passed from the hidden layer to the ouput nodes. the computations are transformations using the weights and bias.
### Output Layer: This is a layer that is responsible for computations and transferring information from the entire neural network and the user. The computations are usually produced between a specified range. 
### Neuron: this is the basic unit of computation in a neural network. also called a node or a unit. Recieves input from an external source and computes an output. 

<img align="center" src="https://ujwlkarn.files.wordpress.com/2016/08/screen-shot-2016-08-09-at-3-42-21-am.png?w=568&h=303" width=600>
<br></br>
<br></br>

  
### Weight: this can be thought of as similar to a coefficient in a linear or logistic regression. It is applied to an input of information. 
### Activation Function: a function that is used to get an output from a node or neuruon. in each layer, not all activation functions are actually triggered. this is how information is filtered between layers. it will define the output of that neuron given a specific set of inputs.
### Node Map: visual diagram that shows the architecture of the neural network. Shows the layers and nodes of the neural netrowk. 
### Perceptron: the simplest form of a neural network. a single layered neural network.


## Inputs -> Outputs

### Explain the flow of information through a neural network from inputs to outputs. Be sure to include: inputs, weights, bias, and activation functions. How does it all flow from beginning to end?

#### Your Answer Here

## Write your own perceptron code that can correctly classify (99.0% accuracy) a NAND gate. 

| x1 | x2 | y |
|----|----|---|
| 0  | 0  | 1 |
| 1  | 0  | 1 |
| 0  | 1  | 1 |
| 1  | 1  | 0 |

In [0]:
import pandas as pd
import numpy as np
data = { 'x1': [0,1,0,1],
         'x2': [0,0,1,1],
         'y':  [1,1,1,0]
       }

df = pd.DataFrame.from_dict(data).astype('int')

NAND = pd.concat([df, df, df, df, df, df, df, df])

In [0]:
class Perceptron(object):
  
  def __init__(self, no_of_inputs, threshold=500, learning_rate=0.01):

    '''
    no_of_inputs: defines how many weights we need to learn
    threshold: number of epochs we'll allow our learning algorithm to iterate thru
    learning_rate: determine the magnitude of change for our weights during each step. 
    '''
    self.threshold = threshold
    self.learning_rate = learning_rate
    self.weights = np.zeros(no_of_inputs + 1) # initialize weight vector. Gives array of zeros equal to num_of_inputs plus 1. 



  def train(self, training_inputs, labels):
    '''
    training_inputs: list of numpy vectors to be used as inputs by the predict method
    lables: numpy array of expected output values for each of the corresponding inputs in inputs_list
        len(training_inputs) == len(labels)
    '''

    for _ in range(self.threshold):
      for inputs, label in zip(training_inputs, labels):
        prediction = self.predict(inputs)
        self.weights[1:] += self.learning_rate * (label - prediction) * inputs # label - prediction is the error. this updates the weights
        self.weights[0] += self.learning_rate * (label - prediction) # this updates the bias. 
      # print(prediction)
      # print(self.weights)
      

  def predict(self, inputs):
    summation = np.dot(inputs, self.weights[1:]) + self.weights[0]  # activation function f(x) = w * x + b
    activated_output = 1 / (1 + np.exp(-summation))
    if activated_output >= .5:
      return 1
    else:
      return 0
    

In [0]:
# instatiate the Perceptron Class with 2 inputs
perceptron = Perceptron(2)

# set the training and the labels data
train = np.array(df[['x1', 'x2']])
labels = np.array(df['y'])

# train the Perceptron
perceptron.train(train, labels)

In [120]:
perceptron.predict(np.array([1,1]))

0

## Implement your own Perceptron Class and use it to classify a binary dataset: 
- [The Pima Indians Diabetes dataset](https://raw.githubusercontent.com/ryanleeallred/datasets/master/diabetes.csv) 

You may need to search for other's implementations in order to get inspiration for your own. There are *lots* of perceptron implementations on the internet with varying levels of sophistication and complexity. Whatever your approach, make sure you understand **every** line of your implementation and what its purpose is.

In [102]:
diabetes = pd.read_csv('https://raw.githubusercontent.com/ryanleeallred/datasets/master/diabetes.csv')
diabetes.head()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0
2,8,183,64,0,0,23.3,0.672,32,1
3,1,89,66,23,94,28.1,0.167,21,0
4,0,137,40,35,168,43.1,2.288,33,1


Although neural networks can handle non-normalized data, scaling or normalizing your data will improve your neural network's learning speed. Try to apply the sklearn `MinMaxScaler` or `Normalizer` to your diabetes dataset. 

In [122]:
from sklearn.preprocessing import MinMaxScaler, Normalizer

feats = list(diabetes)[:-1]

feats

# X = ...

['Pregnancies',
 'Glucose',
 'BloodPressure',
 'SkinThickness',
 'Insulin',
 'BMI',
 'DiabetesPedigreeFunction',
 'Age']

In [121]:
# instatiate the Perceptron Class with 2 inputs
perceptron = Perceptron(len(feats))

# set the training and the labels data
train = np.array(diabetes[feats])
labels = np.array(diabetes['Outcome'])

# train the Perceptron
perceptron.train(train, labels)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:34: RuntimeWarning: overflow encountered in exp


In [123]:
perceptron.predict([6, 148, 72, 35, 0, 33.6, .627, 50])

0

In [124]:
perceptron.predict([1, 85, 66, 29, 0, 26.6, .351, 31])

0

In [0]:
pred = []
for index, row in diabetes[feats][600:].iterrows():
  pred.append((perceptron.predict(row)))

In [136]:
test_sample = diabetes[600:]

test_sample['results'] = pred

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [137]:
test_sample

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome,results
600,1,108,88,19,0,27.1,0.400,24,0,0
601,6,96,0,0,0,23.7,0.190,28,0,1
602,1,124,74,36,0,27.8,0.100,30,0,0
603,7,150,78,29,126,35.2,0.692,54,1,1
604,4,183,0,0,0,28.4,0.212,36,1,1
...,...,...,...,...,...,...,...,...,...,...
763,10,101,76,48,180,32.9,0.171,63,0,1
764,2,122,70,27,0,36.8,0.340,27,0,0
765,5,121,72,23,112,26.2,0.245,30,0,1
766,1,126,60,0,0,30.1,0.349,47,1,0


In [141]:
(test_sample['Outcome'] == test_sample['results']).value_counts(normalize=True)

True     0.571429
False    0.428571
dtype: float64

## Stretch Goals:

- Research "backpropagation" to learn how weights get updated in neural networks (tomorrow's lecture). 
- Implement a multi-layer perceptron. (for non-linearly separable classes)
- Try and implement your own backpropagation algorithm.
- What are the pros and cons of the different activation functions? How should you decide between them for the different layers of a neural network?

In [0]:
##### Update this Class #####

class Perceptron:
    
    def __init__(self, niter = 10):
        self.niter = niter
    
    def __sigmoid(self, x):
        return 1 / (1 + np.exp(-x))
    
    def __sigmoid_derivative(self, x):
        return None

    def fit(self, X, y):
    """Fit training data
    X : Training vectors, X.shape : [#samples, #features]
    y : Target values, y.shape : [#samples]
    """

        # Randomly Initialize Weights
        weights = ...

        for i in range(self.niter):
            # Weighted sum of inputs / weights

            # Activate!

            # Cac error

            # Update the Weights


    def predict(self, X):
    """Return class label after unit step"""
        return None